Set up environment

upload config.py to /content and label_map.pbtxt to Tensorflow/workspace/annotations

In [1]:
%run config.py

In [2]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
  !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 75161, done.
remote: Total 75161 (delta 0), reused 0 (delta 0), pack-reused 75161
Receiving objects: 100% (75161/75161), 580.40 MiB | 31.88 MiB/s, done.
Resolving deltas: 100% (53367/53367), done.


In [3]:
# Install Tensorflow Object Detection (TFOD)
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 4.8 MB/s 
     |███████████████████████████████

In [4]:
!pip install tensorflow --upgrade
!pip uninstall protobuf matplotlib -y
!pip install protobuf==3.19 matplotlib==3.2
!pip install -U opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: protobuf 3.19.4
Uninstalling protobuf-3.19.4:
  Successfully uninstalled protobuf-3.19.4
Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 12.4 MB 21.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.9 MB 1.1 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [5]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 60 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 8s (53.9 MB/s)
(Reading database ... 155639 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155617 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

In [6]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2022-07-11 02:24:45.483872: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0711 02:24:45.760681 140181279467392 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.72s
I0711 02:24:46.203063 140181279467392 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.72s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.61s
I0711 02:24:46.809785 140181279467392 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_cr

In [7]:
# Re-run config.py after restarting the runtime.
%run config.py
import object_detection

Load the trained model

In [8]:
import tarfile
file_model = tarfile.open('/content/models.tar.gz')
file_model.extractall('/content/')  
file_model.close()

In [9]:
!wget {PRETRAINED_MODEL_URL}
!mv {'models.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {'models.tar.gz'}

--2022-07-11 02:25:57--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.213.128, 2607:f8b0:400c:c0a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.213.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  6.39MB/s    in 3.1s    

2022-07-11 02:26:00 (6.39 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

Tensorflow/workspace/models/my_ssd_mobnet/
Tensorflow/workspace/models/my_ssd_mobnet/ckpt-3.index
Tensorflow/workspace/models/my_ssd_mobnet/checkpoint
Tensorflow/workspace/models/my_ssd_mobnet/ckpt-3.data-00000-of-00001
Tensorflow/workspace/models/my_ssd_mobnet/ckpt-2.index
Tensorflow/workspace/mod

In [10]:
!unzip 'train.zip' -d '/content/Tensorflow/workspace/images'
!unzip 'test.zip' -d '/content/Tensorflow/workspace/images'

Archive:  train.zip
   creating: /content/Tensorflow/workspace/images/train/
   creating: /content/Tensorflow/workspace/images/train/Annotations/
  inflating: /content/Tensorflow/workspace/images/train/Annotations/099f8fc2-cd_32.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/0dead746-cd_08.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/2ba4252f-cd_24.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/66eeee46-cd_34.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/7456c9fd-cd_04.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/a6ea9237-cd_07.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/b604a9b2-cd_23.xml  
  inflating: /content/Tensorflow/workspace/images/train/Annotations/ba5fa5e8-cd_09.xml  
   creating: /content/Tensorflow/workspace/images/train/images/
  inflating: /content/Tensorflow/workspace/images/train/images/099f8fc2-cd_32.

In [11]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import pandas as pd

In [12]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [13]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

Generate candy_counter

In [39]:
def get_detection(image):
  img = cv2.imread(image)
  image_np = np.array(img)
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections = detect_fn(input_tensor)

  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  return detections

In [60]:
def get_dataset(detections):
  boxes = detections['detection_boxes']
  classes = detections['detection_classes']
  scores = detections['detection_scores']

  dataset = pd.DataFrame({'class': classes, 'box': list(boxes), 'score': scores}, columns=['class', 'score', 'box'])
  dataset.drop(dataset[dataset['score'] <= 0.75].index, inplace = True)

  duplicate_index_i = []
  duplicate_index_j = []
  for i in range(len(dataset)):
    for j in range(i+1,len(dataset)):
        if (dataset['box'][i] == dataset['box'][j]).all():
          duplicate_index_i.append(i)
          duplicate_index_j.append(j)
  
  data = dataset.drop(duplicate_index_j)
  data_use = data.reset_index()
  return data_use

In [61]:
detection1 = get_detection(image_np)
df1 = get_dataset(detection1)

In [62]:
num_moon = 0
num_insect = 0
num_bstar = 0
num_gstar = 0
num_uniw = 0
num_unih = 0
num_owl = 0
num_cat = 0

In [63]:
df1

,index,class,score,box
0,0,1,0.999914,"[0.08098981, 0.019202255, 0.2293489, 0.1576555]"
1,1,5,0.999372,"[0.57674116, 0.64962244, 0.75246793, 0.7699895]"
2,2,7,0.999127,"[0.38854504, 0.12613563, 0.5542983, 0.21219699]"
3,3,4,0.998062,"[0.4402621, 0.83669037, 0.7220446, 0.94455475]"
4,4,2,0.996948,"[0.21263957, 0.19369788, 0.33399075, 0.28458664]"
5,5,3,0.996768,"[0.25677693, 0.6686455, 0.3744353, 0.7617271]"
6,6,1,0.995038,"[0.4045349, 0.30578735, 0.5811579, 0.40156302]"
7,7,2,0.955514,"[0.16505104, 0.49300304, 0.28566104, 0.5778221]"
8,8,6,0.947914,"[0.12443214, 0.3359583, 0.30181092, 0.40966785]"
9,9,5,0.938850,"[0.42922616, 0.52698344, 0.54344195, 0.7386531]"


In [64]:
for i in range(len(df1)):  
  if df1['class'][i] == 1:
    num_moon += 1
  elif df1['class'][i] == 2:
    num_insect += 1
  elif df1['class'][i] == 3:
    num_bstar += 1  
  elif df1['class'][i] == 4:
    num_gstar += 1
  elif df1['class'][i] == 5:
    num_uniw += 1
  elif df1['class'][i] == 6:
    num_unih += 1
  elif df1['class'][i] == 7:
    num_owl += 1
  elif df1['class'][i] == 8:
    num_cat += 1
  else:
    print('error')

In [65]:
def candy_counter(ipath):
  detection1 = get_detection(ipath)
  df = get_dataset(detection1)
  num_moon = 0
  num_insect = 0
  num_bstar = 0
  num_gstar = 0
  num_uniw = 0
  num_unih = 0
  num_owl = 0
  num_cat = 0

  for i in range(len(df)):
    if df['class'][i] == 1:
      num_moon += 1
    elif df['class'][i] == 2:
      num_insect += 1
    elif df['class'][i] == 3:
      num_bstar += 1  
    elif df['class'][i] == 4:
      num_gstar += 1
    elif df['class'][i] == 5:
      num_uniw += 1
    elif df['class'][i] == 6:
      num_unih += 1
    elif df['class'][i] == 7:
      num_owl += 1
    elif df['class'][i] == 8:
      num_cat += 1
    else:
      print('error')

    result = {'Moon': num_moon,
    'Insect': num_insect,
    'Black_star': num_bstar,
    'Grey_star': num_gstar,
    'Unicorn_whole': num_uniw,
    'Unicorn_head': num_unih,
    'Owl': num_owl,
    'Cat': num_cat}
  return result

In [68]:
imgpath = '/content/Tensorflow/workspace/images/test/images/bef9e5f4-cd_22.jpg'

In [69]:
candy_counter(imgpath)

{'Black_star': 3,
 'Cat': 0,
 'Grey_star': 2,
 'Insect': 3,
 'Moon': 2,
 'Owl': 1,
 'Unicorn_head': 1,
 'Unicorn_whole': 2}